In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

reverse_word_index = None
train_data = None
train_labels = None
test_data = None
test_labels = None

1.12.0


### Get data and create dictionary for word index

In [2]:
def loadDatasets():
    global train_data, train_labels, test_data, test_labels
    imdb = keras.datasets.imdb

    (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000, path=r"E:\Projects\machineLearning\src\mnist\dataset\imdb\imdp.npz")
    print("Training entries:{}, labels:{}".format(len(train_data), len(train_labels)))
    print("train_data.shape:{}\nlength of train_data[0]:{} length of train_data[1]:{}\ntrain_data[0]:{}"
          .format(train_data.shape, len(train_data[0]), len(train_data[1]), train_data[0]))
    
    global reverse_word_index
    # A dictionary mapping words to an integer index
    word_index = imdb.get_word_index()

    # The first indices are reserved
    word_index = {k:(v+3) for k,v in word_index.items()}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2  # unknown
    word_index["<UNUSED>"] = 3

    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [3]:

def decode_review(text):
    global reverse_word_index
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

### Main()

In [4]:
if __name__=='__main__':
    isPrepareDatasetsNeeded = True
    if isPrepareDatasetsNeeded:
        loadDatasets()
    global reverse_word_index
    print(reverse_word_index)
    #decode_review(train_data[0])
    

Training entries:25000, labels:25000
train_data.shape:(25000,)
length of train_data[0]:218 length of train_data[1]:189
train_data[0]:[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb_word_index.json: None -- EOF occurred in violation of protocol (_ssl.c:833)